In [7]:
from readability import Document
import crawl4ai
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [1]:
%%capture
!crawl4ai-setup
!craw4ai-doctor

In [8]:
from crawl4ai.content_filter_strategy import PruningContentFilter
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
from crawl4ai import AsyncWebCrawler, CacheMode, BrowserConfig, CrawlerRunConfig, CacheMode, DefaultMarkdownGenerator, PruningContentFilter
from playwright.async_api import Page, BrowserContext
from IPython.display import display, HTML, Markdown

In [ ]:
async def workflow(verbose=True):
    async with AsyncWebCrawler(config=BrowserConfig(text_mode=True)) as crawler:
        # Set a 'before_goto' hook to run custom code just before navigation
        # crawler.crawler_strategy.set_hook("before_return_html", before_return_html)

        # Perform the crawl operation
        result = await crawler.arun(
            url="https://readhacker.news/s/6nUzH",
            config = CrawlerRunConfig(
            cache_mode=CacheMode.ENABLED,
            excluded_tags=['nav', 'footer', 'aside'],
            remove_overlay_elements=True,
            markdown_generator=DefaultMarkdownGenerator(
                content_filter=PruningContentFilter(threshold=0.48, threshold_type="fixed", min_word_threshold=0),
                options={
                    "ignore_links": True
                }
            ),
        )
        )
        html = result.html
        # html_cleaned_by_crawl4ai = PruningContentFilter()
        html_cleaned_by_readability = Document(html).summary()
        display(Markdown(DefaultMarkdownGenerator().generate_markdown(html).raw_markdown))
        # print(Document(result.html).get_clean_html())

asyncio.run(workflow())

In [2]:
import pandas as pd
from pathlib import Path
from itables import init_notebook_mode, show

# init_notebook_mode(all_interactive=True)

In [ ]:

# Get all CSV files in current directory
csv_files = list(Path('.').glob('*.csv'))

# Read and concatenate all CSV files
dfs = []
for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file)
        df['source'] = csv_file.stem  # Add source column to track origin
        dfs.append(df)
    except Exception as e:
        print(f"Error reading {csv_file}: {e}")

# Combine all dataframes
combined_df = pd.concat(dfs, ignore_index=True)

# Create interactive grid widget
display(HTML(combined_df.head(10).to_html()))

# Example filters (can be run in subsequent cells):
"""
# Filter by source
hn_data = combined_df[combined_df['source'] == 'hn']

# Filter by column value
filtered_df = combined_df[combined_df['views'] > 1000]

# Search text in specific column
search_results = combined_df[combined_df['text'].str.contains('python', case=False, na=False)]
"""
